In [1]:

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.layers import Layer
from keras import backend as K
import time
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
 
import fr_utils
from inception_blocks_v2 import *
 
# %matplotlib inline
# %load_ext autoreload
# %autoreload 2
import sys
np.set_printoptions(threshold=sys.maxsize) 

FRmodel = faceRecoModel(input_shape=(96, 96, 3))


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [3]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    
    
    #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    
    #第三步：减去之前的两个距离，然后加上alpha
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    
    
    #通过取带零的最大值和对训练样本的求和来计算整个公式
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss


In [4]:
with tf.compat.v1.Session() as test:
    tf.compat.v1.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 528.1426


2022-06-29 22:44:43.946211: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
database = {}
# database["danielle"] = fr_utils.img_to_encoding("images/danielle.png", FRmodel)
# database["younes"] = fr_utils.img_to_encoding("images/younes.jpg", FRmodel)
# database["tian"] = fr_utils.img_to_encoding("images/tian.jpg", FRmodel)
# database["andrew"] = fr_utils.img_to_encoding("images/andrew.jpg", FRmodel)
# database["kian"] = fr_utils.img_to_encoding("images/kian.jpg", FRmodel)
# database["dan"] = fr_utils.img_to_encoding("images/dan.jpg", FRmodel)
# database["sebastiano"] = fr_utils.img_to_encoding("images/sebastiano.jpg", FRmodel)
# database["bertrand"] = fr_utils.img_to_encoding("images/bertrand.jpg", FRmodel)
# database["kevin"] = fr_utils.img_to_encoding("images/kevin.jpg", FRmodel)
# database["felix"] = fr_utils.img_to_encoding("images/felix.jpg", FRmodel)
# database["benoit"] = fr_utils.img_to_encoding("images/benoit.jpg", FRmodel)
# database["arnaud"] = fr_utils.img_to_encoding("images/arnaud.jpg", FRmodel)
from PIL import Image
for filename in os.listdir(r"./images"):
    img = Image.open("images/" + filename)
    img = img.resize((96,96),Image.ANTIALIAS)
    img.save("images/"+filename)
database["0001"]=fr_utils.img_to_encoding("images/00002_940928_fb.png", FRmodel)

/Users/zhuyouyu/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [6]:
# GRADED FUNCTION: verify

def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    ### START CODE HERE ###
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = fr_utils.img_to_encoding(image_path, model)
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding - database[identity])
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    ### END CODE HERE ###
        
    return dist, door_open

In [7]:
for filename in os.listdir(r"./images"):
    verify("images/"+filename, "0001", database, FRmodel)

It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!
It's 0001, welcome in!


In [8]:
# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = fr_utils.img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for name, db_enc in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(db_enc - encoding)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [9]:
for filename in os.listdir(r"./images"):
    who_is_it("images/"+filename, database, FRmodel)

it's 0001, the distance is 0.15258802
it's 0001, the distance is 0.18391225
it's 0001, the distance is 0.0
it's 0001, the distance is 0.05489888
it's 0001, the distance is 0.0464016
it's 0001, the distance is 0.039007634
it's 0001, the distance is 0.08048336
it's 0001, the distance is 0.058319047
it's 0001, the distance is 0.09002686
it's 0001, the distance is 0.070436835
it's 0001, the distance is 0.042739272
it's 0001, the distance is 0.054926787
it's 0001, the distance is 0.15668063
it's 0001, the distance is 0.15709238
it's 0001, the distance is 0.15402456
